In [ ]:
pip install lightgbm -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from lightgbm import LGBMRegressor  
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import shap
from sklearn.impute import SimpleImputer

data = pd.read_excel(r"I:\1\DataSet.xlsx")

X = data.drop(columns=["V"])
y = data["V"]  

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

X_imputed[np.isinf(X_imputed)] = np.nan
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X_imputed)

X = pd.DataFrame(X_imputed, columns=X.columns)

lgbm = LGBMRegressor()  

param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [-1, 5, 10]
}

cv = KFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X, y)

def custom_cross_val_score(model, X, y, cv, scoring):
    performance_records = []
    for train_index, test_index in cv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        performance_records.append({'RMSE': rmse, 'R2': r2, 'MAE': mae, 'MAPE': mape})
    return pd.DataFrame(performance_records)

performance_df_cv = custom_cross_val_score(lgbm, X, y, cv, scoring='neg_mean_squared_error')

performance_df_cv.to_excel(r'I:\1\LGBM-Ten-fold cross validation performance metrics.xlsx', index=False)

grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X, y)

best_params = grid_search.best_params_

best_params_df = pd.DataFrame([best_params])

best_params_df.to_excel(r'I:\1\LGBM-optimal parameter.xlsx', index=False)

lgbm_best = LGBMRegressor(**best_params)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lgbm_best.fit(X_train, y_train)

y_pred = lgbm_best.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

performance_df = pd.DataFrame({
    'RMSE': [rmse],
    'R2': [r2],
    'MAE': [mae],
    'MAPE': [mape]
})
performance_df.to_excel(r'I:\1\LGBM-Performance indicators.xlsx', index=False)

explainer = shap.Explainer(lgbm_best)

shap_values = explainer(X_train)

shap_df = pd.DataFrame(shap_values.values, columns=X_train.columns)

shap_mean = shap_df.abs().mean().sort_values(ascending=False)

shap_mean_df = pd.DataFrame(shap_mean, columns=['Average SHAP Value'])

shap_mean_df.to_excel(r'I:\1\LGBM-SHAP.xlsx')

shap.summary_plot(shap_values, X_train)

shap.summary_plot(shap_values, X_train, plot_type="bar")

predictions_df = pd.DataFrame({'actual value': y_test, 'predicted value': y_pred})
predictions_df.to_excel(r'I:\1\LGBM-Training results.xlsx')

print("All steps have been completed and the relevant files have been exported.")

In [ ]:
new_data = pd.read_excel(r'I:\1\Data for prediction.xlsx')

X_new = new_data.drop(columns=["V"])

imputer = SimpleImputer(strategy='median')
X_new_imputed = imputer.fit_transform(X_new)

X_new_imputed[np.isinf(X_new_imputed)] = np.nan
imputer = SimpleImputer(strategy='median')
X_new_imputed = imputer.fit_transform(X_new_imputed)

X_new = pd.DataFrame(X_new_imputed, columns=X_new.columns)

y_new_pred = lgbm_best.predict(X_new)

predictions_new_df = pd.DataFrame({'Predicted': y_new_pred})
predictions_new_df.to_excel(r'I:\1\new results.xlsx', index=False)

print("Predictions for new data have been saved.")